In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC


In [2]:
from typing_extensions import assert_type
#getting files
sample = pd.read_csv('sample_submission.csv')
X_test = pd.read_csv('test.csv')
df = pd.read_csv('train.csv')


df['train_test'] = 1
X_test['train_test'] = 0
all_data = pd.concat([df, X_test])

In [3]:
all_data.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status,train_test
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0.0,1
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0.0,1
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0.0,1
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0.0,1
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0.0,1


In [4]:
all_data['loan_grade'] = all_data['loan_grade'].apply(lambda x: 1 if x == 'A' else 2 if x == 'B' or 'C' else 3)
all_data['cb_person_cred_hist_length'] = all_data['cb_person_cred_hist_length'].apply(lambda x: 1 if x <2 else 2 if x < 4 else 3 if x <7 else 4)
all_data['overqualifed'] = all_data['person_income'].apply(lambda x: 1 if x > 100000 else 0)
all_data['best_target'] = all_data['person_age'].apply(lambda x: 1 if x > 35 else 0)

In [5]:
#check nulls values in data
missing_values = all_data.isnull().sum()
print(missing_values)

id                                0
person_age                        0
person_income                     0
person_home_ownership             0
person_emp_length                 0
loan_intent                       0
loan_grade                        0
loan_amnt                         0
loan_int_rate                     0
loan_percent_income               0
cb_person_default_on_file         0
cb_person_cred_hist_length        0
loan_status                   39098
train_test                        0
overqualifed                      0
best_target                       0
dtype: int64


In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

object_cols = ['person_home_ownership','loan_intent', 'cb_person_default_on_file']

label_encoder = LabelEncoder()

for col in object_cols:
    # Convert all values to strings before fitting the encoder
    all_values = pd.concat([all_data[col]]).astype(str).unique()
    label_encoder.fit(all_values)

    all_data[col] = label_encoder.transform(all_data[col].astype(str))

In [7]:
Y_train = all_data[all_data.train_test == 1]['loan_status']
X_train1 = all_data[all_data.train_test == 1].drop(['train_test','loan_status'], axis =1)
X_test1 = all_data[all_data.train_test == 0].drop(['train_test','loan_status'], axis =1)


In [8]:
#importing ml models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import HistGradientBoostingClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV # import GridSearchCV

In [9]:
# model1 = HistGradientBoostingClassifier()
# model1.fit(X_train1, Y_train)
# model3 = DecisionTreeClassifier()
# model3.fit(X_train1, Y_train)
# model6 = KNeighborsClassifier(n_neighbors=15)
# model6.fit(X_train1, Y_train)
# from sklearn.ensemble import RandomForestClassifier
# model7 = RandomForestClassifier(n_estimators=5000, random_state=42)
# model7.fit(X_train1, Y_train)

In [10]:
from lightgbm import LGBMClassifier
study_LGBM = {
        'metric': 'auc',
        'learning_rate': 0.05,
        'num_leaves': 25,
        'max_depth': 31,
        'min_data_in_leaf': 20,
        'max_bin': 255,
        'verbose': -1,
        'seed': 42
    }

#Best parameters found by grid search are:
#study_LGBM = {'colsample_bytree': 0.4637666516858401, 'min_child_samples': 283, 'min_child_weight': 0.01, 'num_leaves': 6, 'reg_alpha': 5, 'reg_lambda': 20, 'subsample': 0.7267387414492084}

In [11]:
!pip install catboost
import xgboost as xgb
import catboost as cb
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint, uniform

#params for xgb
param_distributions = {
    'n_estimators': randint(100, 200),  # Sample integers between 100 and 500
    'learning_rate': uniform(0.01, 0.2),  # Sample floats between 0.01 and 0.2
    'max_depth': randint(3, 10),
    'subsample': uniform(0.7, 0.3),  # Sample floats between 0.7 and 1.0 (0.7 + 0.3)
    'colsample_bytree': uniform(0.7, 0.3),
    'gamma': uniform(0, 0.5)
    # 'reg_alpha': uniform(0, 0.5),
    # 'reg_lambda': uniform(0, 0.5)
}

# random_search.fit(X_train1, Y_train)
# print("Best parameters:", random_search.best_params_)
# print("Best model:", random_search.best_estimator_)

# best parameters below with sample of 500 n_iter
#Best parameters: {'colsample_bytree': 0.7360493943416926, 'gamma': 0.1709398333582008, 'learning_rate': 0.028359813162688374, 'max_depth': 9, 'n_estimators': 193, 'reg_alpha': 0.4277302920055036, 'reg_lambda': 0.35182892969001184, 'subsample': 0.8422521487261975}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.8 MB/s eta 0:00:00


In [12]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train1)
X_train_scaled = scaler.transform(X_train1)
X_test_scaled = scaler.transform(X_test1)

In [13]:
xgb_best = {'colsample_bytree': 0.7360493943416926, 'gamma': 0.1709398333582008, 'learning_rate': 0.028359813162688374, 'max_depth': 9, 'n_estimators': 193, 'reg_alpha': 0.4277302920055036, 'reg_lambda': 0.35182892969001184, 'subsample': 0.8422521487261975}

model5 = xgb.XGBClassifier(**xgb_best)
model5.fit(X_train1, Y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.7360493943416926, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0.1709398333582008,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.028359813162688374,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=193, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [14]:
#finding best parameters for catboost

In [15]:
!pip install catboost==1.2.2



cat_boost = cb.CatBoostClassifier(
    iterations=2100,
    max_depth=7,
    learning_rate=0.04397761414212552,
    l2_leaf_reg=7.138769102913285,
    border_count=68,
    random_strength=7.6509581572602725,
    bagging_temperature=0.9863297723963542,
    colsample_bylevel=0.29574049881294256,
    min_data_in_leaf=9,
    random_state=42,
    early_stopping_rounds=200,
    # Remove use_best_model or provide eval_set
    # use_best_model=True, # Remove this line if you don't have an eval_set
    allow_writing_files=False,
    task_type="CPU",
    loss_function='Logloss',
    eval_metric="AUC",
)


# Train the model
# Provide an eval_set if you want to use use_best_model=True
# For example:
# cat_boost.fit(X_train1, Y_train, eval_set=(X_test1, Y_test))
cat_boost.fit(X_train1, Y_train)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: catboost
    Found existing installation: catboost 1.2.7
    Uninstalling catboost-1.2.7:
      Successfully uninstalled catboost-1.2.7


0:	total: 84.7ms	remaining: 2m 57s
1:	total: 115ms	remaining: 2m 1s
2:	total: 158ms	remaining: 1m 50s
3:	total: 208ms	remaining: 1m 48s
4:	total: 237ms	remaining: 1m 39s
5:	total: 263ms	remaining: 1m 31s
6:	total: 292ms	remaining: 1m 27s
7:	total: 321ms	remaining: 1m 23s
8:	total: 356ms	remaining: 1m 22s
9:	total: 383ms	remaining: 1m 20s
10:	total: 395ms	remaining: 1m 14s
11:	total: 407ms	remaining: 1m 10s
12:	total: 427ms	remaining: 1m 8s
13:	total: 439ms	remaining: 1m 5s
14:	total: 449ms	remaining: 1m 2s
15:	total: 461ms	remaining: 1m
16:	total: 473ms	remaining: 58s
17:	total: 485ms	remaining: 56.1s
18:	total: 498ms	remaining: 54.6s
19:	total: 511ms	remaining: 53.1s
20:	total: 540ms	remaining: 53.5s
21:	total: 564ms	remaining: 53.3s
22:	total: 577ms	remaining: 52.1s
23:	total: 604ms	remaining: 52.3s
24:	total: 629ms	remaining: 52.2s
25:	total: 653ms	remaining: 52.1s
26:	total: 675ms	remaining: 51.8s
27:	total: 701ms	remaining: 51.9s
28:	total: 734ms	remaining: 52.4s
29:	total: 756ms	

In [16]:
#cross validation with catboost

In [17]:
# Modify LGBMClassifier parameters
best_LGBM = LGBMClassifier(**study_LGBM, class_weight='balanced', random_state=42) # Change the device parameter to 'cpu'
best_LGBM.fit(X_train1, Y_train)

LGBMClassifier(class_weight='balanced', learning_rate=0.05, max_bin=255,
               max_depth=31, metric='auc', min_data_in_leaf=20, num_leaves=25,
               random_state=42, seed=42, verbose=-1)

In [18]:
#ensemble = VotingClassifier(estimators=[('hg1', model1),('dt1', model3),('xg1',model5), ('kn', model6),('rf', model7),('lgbm', best_LGBM)], voting='hard')
# ensemble = VotingClassifier(estimators=[('cb',cat_boost),('lgbm', best_LGBM),('xg1',model5)], voting='hard')

In [19]:
# ensemble.fit(X_train1, Y_train )

In [20]:
y_pred = cat_boost.predict(X_test1)

In [21]:
submission = pd.DataFrame({
    'id': X_test1['id'],
    'loan_status': y_pred.astype(int)  # Convert predictions to integers
})

In [22]:
submission.to_csv('submission3.csv', index=False)